In [1]:
#@title Install required libraries
!pip -q install transformers accelerate sentence-transformers --upgrade
!pip -q install python-docx pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 78.2 MB/s eta 0:00:00


In [2]:
#@title Runtime config
EMBED_MODEL_NAME    = "sentence-transformers/paraphrase-MiniLM-L6-v2" #@param ["sentence-transformers/paraphrase-MiniLM-L6-v2", "intfloat/multilingual-e5-base", "BAAI/bge-m3"] {allow-input: true}
LLM_MODEL_NAME      = "K-intelligence/Midm-2.0-Base-Instruct"
MAX_NEW_TOKENS_LLM  = 900   # 필요에 따라 조정

print("LLM :", LLM_MODEL_NAME)
print("EMB :", EMBED_MODEL_NAME)


LLM : K-intelligence/Midm-2.0-Base-Instruct
EMB : sentence-transformers/paraphrase-MiniLM-L6-v2


In [3]:
# 파일: testbench_w_model (1).ipynb
# 역할: HFLLMAdapter 클래스를 수정하여 Completion Mode를 강제합니다.

from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import torch
import re
from typing import List

class HFLLMAdapter:
    """
    K-intelligence/Midm-2.0-Base-Instruct 모델과 호환되며,
    Chat Mode 대신 Completion Mode 사용을 강제하도록 수정된 버전입니다.
    """
    def __init__(self, model_name: str = "K-intelligence/Midm-2.0-Base-Instruct", max_new_tokens: int = 900):
        self.model_name = model_name
        self.tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
            trust_remote_code=True,
        )
        self.max_new_tokens = max_new_tokens

        if self.model.config.pad_token_id is None and self.tok.eos_token_id is not None:
            self.model.config.pad_token_id = self.tok.eos_token_id

    def _format_input(self, prompt: str) -> str:
        """
        [수정된 부분]
        Chat Template을 사용하지 않고 항상 원본 프롬프트를 그대로 반환하여
        Completion Mode로 작동하도록 강제합니다.
        """
        return prompt

    def complete(self, prompt: str, **gen_kwargs) -> str:
        input_text = self._format_input(prompt)
        inputs = self.tok(input_text, return_tensors="pt")

        # Midm 모델이 사용하지 않는 token_type_ids 제거
        inputs.pop("token_type_ids", None)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        # `temperature` 등 Completion Mode와 관련 없는 인자는 제거합니다.
        # 이 모델 어댑터는 결정론적(deterministic) 출력을 위해 do_sample=False를 사용합니다.
        gen_kwargs.pop('temperature', None)
        gen_kwargs.pop('top_p', None)
        gen_kwargs.pop('top_k', None)

        out = self.model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=gen_kwargs.get("max_tokens", self.max_new_tokens),
            eos_token_id=self.tok.eos_token_id,
            pad_token_id=self.model.config.pad_token_id,
            **gen_kwargs,
        )

        # 입력 프롬프트를 제외한 순수 생성 부분만 디코딩합니다.
        output_tokens = out[0][len(inputs["input_ids"][0]):]
        text = self.tok.decode(output_tokens, skip_special_tokens=True)

        return text.strip()


class HFSentenceEmbedAdapter:
    """
    임베딩 모델을 위한 어댑터입니다. (변경 없음)
    """
    def __init__(self, model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def embed(self, texts: List[str], **kwargs) -> List[List[float]]:
        embs = self.model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        return [e.tolist() for e in embs]

In [13]:
#@title Upload documents to ingest (txt, docx, pdf, hwpx 등)
from google.colab import files
from pathlib import Path

print("인제스트할 문서 파일들을 선택하세요.")
uploaded_docs = files.upload()  # 여러 개 선택 가능
doc_paths = []
for name, data in uploaded_docs.items():
    p = Path(name)
    with open(p, "wb") as f:
        f.write(data)
    doc_paths.append(str(p.resolve()))

print("✅ 업로드 완료:", doc_paths)


인제스트할 문서 파일들을 선택하세요.


Saving sample_2.txt to sample_2.txt
✅ 업로드 완료: ['/content/sample_2.txt']


In [5]:
# 모델 초기화
llm = HFLLMAdapter(LLM_MODEL_NAME, max_new_tokens=MAX_NEW_TOKENS_LLM)
embedder = HFSentenceEmbedAdapter(EMBED_MODEL_NAME)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#@title Run full pipeline on uploaded documents
import importlib, json, time
from pathlib import Path

file_reader   = importlib.import_module("file_reader")
text_chunker  = importlib.import_module("text_chunker")
embedding_mod = importlib.import_module("embedding")

In [14]:


all_records = []
t0 = time.time()

for path in doc_paths:
    print(f"\n=== Processing: {path} ===")
    # 1) 파일 로드
    try:
        text = file_reader.read_file(path)
    except Exception as e:
        print(f"[ERR] read_file failed: {e}")
        continue

    if not isinstance(text, str) or not text.strip():
        print("[WARN] Empty text; skipping.")
        continue
    print(f"[OK] Loaded text: {len(text)} chars")

    # 2) LLM 기반 청크 분할
    try:
        chunk_result = text_chunker.llm_guided_sentence_chunk(
            text,
            llm=llm,
            max_sentences_per_chunk=20,
            system_hint="논리/주제 전환 기준으로 5~20문장씩 묶고, 오직 JSON만 출력하세요."
        )
    except Exception as e:
        print(f"[ERR] chunking failed: {e}")
        continue

    if not hasattr(chunk_result, "chunks") or len(chunk_result.chunks) == 0:
        print("[WARN] No chunks; skipping.")
        continue

    print(f"[OK] Chunks: {len(chunk_result.chunks)}")

    # 3) 임베딩
    chunks = [c.text for c in chunk_result.chunks]
    offsets = [{
        "chunk_index": c.chunk_index,
        "char_start": c.char_start,
        "char_end": c.char_end,
        "word_start": c.word_start,
        "word_end": c.word_end
    } for c in chunk_result.chunks]

    try:
        records = embedding_mod.build_embeddings(
            chunks=chunks,
            file_path=path,
            filetype=Path(path).suffix.lstrip(".").lower() or "unknown",
            offsets=offsets,
            embedder=embedder
        )
    except Exception as e:
        print(f"[ERR] embedding failed: {e}")
        continue

    print(f"[OK] Embeddings: {len(records)} vectors, dim={len(records[0].vector) if records else 0}")
    all_records.extend(records)

t1 = time.time()
print(f"\n=== DONE: {len(all_records)} records from {len(doc_paths)} files in {t1-t0:.2f}s ===")

# 4) 저장 (JSONL)
out_path = Path("embeddings.jsonl")
with open(out_path, "w", encoding="utf-8") as f:
    for r in all_records:
        f.write(json.dumps({
            "vector": r.vector,
            "text": r.text,
            "meta": {
                "file_path": r.meta.file_path,
                "filetype": r.meta.filetype,
                "chunk_index": r.meta.chunk_index,
                "char_start": r.meta.char_start,
                "char_end": r.meta.char_end,
                "word_start": r.meta.word_start,
                "word_end": r.meta.word_end
            }
        }, ensure_ascii=False) + "\n")

print(f"📝 Saved embeddings to: {out_path.resolve()}")



=== Processing: /content/sample_2.txt ===
[OK] Loaded text: 1325 chars
=== RAW LLM OUTPUT (first try) ===
{... }

{
  "chunks": [
    {
      "first_sentence": "원격 근무는 더 이상 일부 기술 기업의 전유물이 아닌, 전 세계적인 업무 패러다임의 전환을 이끌고 있습니다.",
      "last_sentence": "기업 입장에서도 혜택은 명확합니다."
    },
    {
      "first_sentence": "물리적인 사무실 운영 비용을 절감할 수 있으며, 지역에 구애받지 않고 전 세계의 우수한 인재를 채용할 수 있는 기회를 얻게 되었습니다.",
      "last_sentence": "하지만 원격 근무의 단점 또한 명확하게 드러났습니다."
    },
    {
      "first_sentence": "많은 직원이 동료와의 물리적 단절로 인한 소외감과 고립감을 호소하고 있습니다.",
      "last_sentence": "성공적인 하이브리드 근무 모델을 구축하기 위해서는 기술적 기반이 필수적입니다."
    },
    {
      "first_sentence": "클라우드 기반의 협업 도구, 원활한 화상 회의 시스템, 강력한 보안 솔루션 없이는 원활한 운영이 불가능합니다.",
      "last_sentence": "결론적으로, 미래의 업무 환경은 하나의 정답이 있는 것이 아니라 각 기업의 특성과 상황에 맞는 최적의 균형점을 찾는 과정이 될 것입니다."
    },
    {
      "first_sentence": "이러한 변화는 단순히 일하는 장소의 이동을 넘어, 조직 문화와 기술 인프라 전반의 혁신을 요구합니다.",
      "last_sentence": "이러한 변화는 단순한 근무 형태의 변화를 넘어, 일의 본질과 조직의 역할에 대한 근본적인 성찰을 요구하고 있습니다."
    }
  ]
}
===

In [12]:
#@title (Optional) Preview a few records
import json
from pathlib import Path
from itertools import islice

path = Path("embeddings.jsonl")
if not path.exists():
    print("embeddings.jsonl 이 없습니다. 이전 셀을 먼저 실행하세요.")
else:
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(islice(f, 5)):
            row = json.loads(line)
            preview = row["text"][:120].replace("\n", " ")
            print(f"[{i}] {row['meta']['filetype']} #{row['meta']['chunk_index']} | {preview}")


[0] txt #0 | 인공지능은 이제 더 이상 연구실 안의 기술이 아닙니다. 우리는 매일같이 인공지능이 적용된 제품과 서비스를 사용하고 있습니다. 스마트폰의 얼굴 인식 기능, 온라인 쇼핑의 맞춤형 추천, 내비게이션의 교통 예측까지 모두 
[1] txt #1 | 어떤 사람들은 인공지능에 익숙하지만, 다른 사람들은 여전히 기술 적응에 어려움을 겪습니다. 따라서 교육과 지원 정책은 필수적입니다. 하지만 인공지능의 발전이 긍정적인 효과만을 가져오는 것은 아닙니다. 자동화로 인해 
[2] txt #2 | 인공지능을 관리하고, 데이터를 분석하며, 윤리를 검토하는 직업은 앞으로 수요가 더 커질 것입니다. 하지만 전환 과정에서 겪게 될 사회적 비용은 무시할 수 없습니다. 따라서 사회 안전망을 강화하고, 재교육 시스템을 체
[3] txt #3 | 또한 범죄 예측 시스템이나 신용 평가 시스템에서도 편향이 발견되었습니다. 이러한 문제는 기술 그 자체의 문제가 아니라, 사회적 맥락이 반영된 결과입니다. 따라서 인공지능의 설계 단계에서부터 편향을 줄이는 노력이 필요
[4] txt #4 | 미국과 중국은 인공지능을 차세대 전략 기술로 보고 막대한 투자를 하고 있습니다. 유럽 역시 데이터 규제와 윤리적 기준을 통해 독자적인 길을 가려 하고 있습니다. 국가 간의 경쟁은 단순한 기술 경쟁을 넘어 정치적 영향
